In [ ]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")
job = Job(glueContext)

In [ ]:
df = spark.read.option("header", True).format("csv").load("s3://BUCKET/glue/data/csv/people/export.csv")
df=df.filter(df.state == "IN")
df.explain()

In [ ]:
csv_dynamic_frame = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    format_options={"header": True},
    connection_options={"paths": ["s3://BUCKET/glue/data/csv/people/export.csv"]},
    format="csv"
)

filtered_dynamic_frame = csv_dynamic_frame.filter(
    lambda x: x["state"] == "IN"
)

filtered_dynamic_frame.toDF().explain()

In [ ]:
dynamic_frame_frm_catalog = glueContext.create_dynamic_frame.from_catalog(
    database="default",
    table_name="pushdown_people"
)

filtered_dynamic_frame_frm_catalog = dynamic_frame_frm_catalog.filter(
    lambda x: x["state"] == "IN"
)

filtered_dynamic_frame_frm_catalog.toDF().explain()